In [ ]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS, Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

### **Loading data**

In [2]:
data_loader = TextLoader('text_data.txt')
loaded_data = data_loader.load()
loaded_data

[Document(metadata={'source': 'text_data.txt'}, page_content='Philosophy is the study of fundamental questions about life, existence, knowledge, values, reason, and reality. The word itself comes from the Greek roots *philo* (love) and *sophia* (wisdom), meaning "love of wisdom." Philosophers explore big questions like "What is truth?", "What is a good life?", and "Do we have free will?" through careful reasoning and debate. Over time, philosophy has branched into areas like ethics (how we should act), metaphysics (what exists), epistemology (how we know things), and logic (how we reason). It’s not just abstract thinking—philosophy helps shape how we understand science, politics, art, and even our daily decisions. Philosophy branches out into several key areas, each exploring different aspects of human thought and experience. **Metaphysics** asks what exists and what it means for something to be real. **Epistemology** dives into how we know what we know—what counts as knowledge, and ho

### **Data splitting**

In [3]:
data_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=40)
splitted_data = data_splitter.split_documents(loaded_data)
splitted_data

[Document(metadata={'source': 'text_data.txt'}, page_content='Philosophy is the study of fundamental questions about life, existence, knowledge, values, reason,'),
 Document(metadata={'source': 'text_data.txt'}, page_content='existence, knowledge, values, reason, and reality. The word itself comes from the Greek roots'),
 Document(metadata={'source': 'text_data.txt'}, page_content='word itself comes from the Greek roots *philo* (love) and *sophia* (wisdom), meaning "love of'),
 Document(metadata={'source': 'text_data.txt'}, page_content='and *sophia* (wisdom), meaning "love of wisdom." Philosophers explore big questions like "What is'),
 Document(metadata={'source': 'text_data.txt'}, page_content='explore big questions like "What is truth?", "What is a good life?", and "Do we have free will?"'),
 Document(metadata={'source': 'text_data.txt'}, page_content='life?", and "Do we have free will?" through careful reasoning and debate. Over time, philosophy has'),
 Document(metadata={'source'

In [ ]:
# Embedding
ollama_embedding = OllamaEmbeddings(model='nomic-embed-text:latest')
ollama_embedding

C:\Users\BW\AppData\Local\Temp\ipykernel_10192\2020651892.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  ollama_embedding = OllamaEmbeddings(model='nomic-embed-text:latest')


OllamaEmbeddings(base_url='http://localhost:11434', model='nomic-embed-text:latest', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [5]:
# Testing embedding
embedded_user_query = ollama_embedding.embed_query('Why critical thinking is important ?')
embedded_user_query

[0.8848803639411926,
 1.39000403881073,
 -2.4527132511138916,
 -0.34019437432289124,
 2.195199728012085,
 0.9820963740348816,
 1.0690025091171265,
 0.8705623149871826,
 0.5724363923072815,
 -0.06082867458462715,
 0.001626059878617525,
 -0.5990268588066101,
 1.6372355222702026,
 1.2854965925216675,
 0.6792675256729126,
 -0.15141870081424713,
 -0.5820764303207397,
 -1.0742512941360474,
 0.11629015952348709,
 0.762637734413147,
 -1.1710596084594727,
 -0.8065813183784485,
 0.14237527549266815,
 -0.19012583792209625,
 1.1526190042495728,
 0.890016496181488,
 -0.1352381557226181,
 -0.7429888844490051,
 -1.9377267360687256,
 1.9784349203109741,
 1.1558736562728882,
 -1.8960793018341064,
 -1.0555070638656616,
 -1.5896458625793457,
 -0.9463412165641785,
 -1.4923428297042847,
 1.4656906127929688,
 0.47819504141807556,
 0.3814391791820526,
 -0.9143674969673157,
 -0.1944580227136612,
 0.5796698331832886,
 0.5624661445617676,
 -1.4221330881118774,
 1.019237995147705,
 0.16919636726379395,
 -0.83782

In [ ]:
# 768 dimensions
len(embedded_user_query)

768

### **Vector database Faiss**

In [8]:
vector_db_faiss = FAISS.from_documents(documents=splitted_data, embedding=ollama_embedding)
vector_db_faiss.save_local("./vector_db_faiss")

In [9]:
retriever = vector_db_faiss.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A281329120>, search_kwargs={})

In [ ]:
# similarity search
vector_db_faiss.similarity_search('what is the importance of philosophy ?', k=3)

[Document(id='c3f64963-7413-4ddf-9d01-fcfce94f665b', metadata={'source': 'text_data.txt'}, page_content='helps shape how we understand science, politics, art, and even our daily decisions. Philosophy'),
 Document(id='17efa8ef-044c-4ee6-8f11-72169fa182e7', metadata={'source': 'text_data.txt'}, page_content='Philosophy is the study of fundamental questions about life, existence, knowledge, values, reason,'),
 Document(id='2b113e75-d2f1-43ba-8363-61068387c6be', metadata={'source': 'text_data.txt'}, page_content='(how we reason). It’s not just abstract thinking—philosophy helps shape how we understand science,')]

In [14]:
# Chat prompt template
prompt = ChatPromptTemplate.from_template("You are a philosopher. use this context: {context} and question: {input}")

In [15]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='You are a philosopher. use this context: {context} and question: {input}'), additional_kwargs={})])

In [16]:
# Ollama model(gemma)
llm = Ollama(model='gemma3:1b')
llm

C:\Users\BW\AppData\Local\Temp\ipykernel_10192\1677603973.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model='gemma3:1b')


Ollama(model='gemma3:1b')

In [17]:
doc_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
doc_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='You are a philosopher. use this context: {context} and question: {input}'), additional_kwargs={})])
| Ollama(model='gemma3:1b')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [18]:
retrieval_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=doc_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A281329120>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='You are a philosopher. use this context: {context} and question: {input}'), additional_kwargs={})])
            | Ollama(model='gem

In [ ]:
# LLm's answer
llm_response = retrieval_chain.invoke({"input": "what is the importance of philosophy ?"})
llm_response

{'input': 'what is the importance of philosophy ?',
 'context': [Document(id='c3f64963-7413-4ddf-9d01-fcfce94f665b', metadata={'source': 'text_data.txt'}, page_content='helps shape how we understand science, politics, art, and even our daily decisions. Philosophy'),
  Document(id='17efa8ef-044c-4ee6-8f11-72169fa182e7', metadata={'source': 'text_data.txt'}, page_content='Philosophy is the study of fundamental questions about life, existence, knowledge, values, reason,'),
  Document(id='2b113e75-d2f1-43ba-8363-61068387c6be', metadata={'source': 'text_data.txt'}, page_content='(how we reason). It’s not just abstract thinking—philosophy helps shape how we understand science,'),
  Document(id='f53638c7-a6fe-4c33-9a34-05040ebd1a76', metadata={'source': 'text_data.txt'}, page_content='values in general, including both ethics and aesthetics. Philosophy doesn’t just live in books or')],
 'answer': "Alright. Let’s consider this. You’ve given me a starting point – a description of philosophy as a

In [21]:
llm_response['answer']

"Alright. Let’s consider this. You’ve given me a starting point – a description of philosophy as a foundational discipline, not just a pursuit of abstract contemplation. And that’s a remarkably astute observation. It’s easy to see philosophy as a solitary exercise, a quiet contemplation. But I believe it’s inextricably linked to the very fabric of human experience – to how we understand *everything*. \n\nLet’s unpack this, and I think we can start to see how philosophy isn't just a field of inquiry, but a framework for navigating the world around us, and within ourselves. \n\nHere's how I see the implications of your framing:\n\n**1. Shaping Science:**  Science, at its core, strives to understand the *how* – the mechanisms, the laws, the demonstrable truths. But philosophy offers a critical lens. It challenges our assumptions *about* those truths. Consider:  what *is* reality?  Is it simply matter and energy, or is there a more fundamental, underlying reality that science is still atte